# Generate your predictions on test data and save the results in required format

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define the ResNet model
class Blue(nn.Module):
    def __init__(self, equal_map):
        super(Blue, self).__init__()
        self.equal_map = equal_map
        if equal_map == True:
            stride = 1
            self.conv1 = nn.Conv2d(16, 16, 3, stride = stride, padding = 1)
        else:
            stride = 2
            self.conv1 = nn.Conv2d(8, 16, 3, stride = stride, padding = 1)
        self.conv2 = nn.Conv2d(16, 16, 3, stride = 1, padding = 1)
        
        if equal_map == False:
            self.proj = nn.Conv2d(8, 16, 1, stride = stride, padding = 1)
        self.bn1 = nn.BatchNorm2d(16)
    def forward(self, x):
        def swish(pre):
            return pre * F.sigmoid(pre)

        out = self.conv1(x)
        # out = F.relu(self.bn1(out))
        out = swish(self.bn1(out))
        out = self.conv2(out)

        if self.equal_map == False:
            x = self.proj(x)

        out = x + out
        # out = F.relu(out)
        out = swish(out)
        return out
class Green(nn.Module):
    def __init__(self, equal_map):
        super(Green, self).__init__()
        self.equal_map = equal_map
        if equal_map == True:
            self.conv1 = nn.Conv2d(32, 32, 3, stride = 1, padding = 1)

        else:
            self.conv1 = nn.Conv2d(16, 32, 3, stride = 2, padding = 1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride = 1, padding = 1)
        
        if self.equal_map == False:
            self.proj = nn.Conv2d(16, 32, 1, stride = 2)
        self.bn1 = nn.BatchNorm2d(32)
    def forward(self, x):
        def swish(pre):
            return pre * F.sigmoid(pre)
        out = self.conv1(x)
        # out = F.relu(self.bn1(out))
        out = swish(self.bn1(out))
        out = self.conv2(out)

        if self.equal_map == False:
            x = self.proj(x)

        out = x + out
        # out = F.relu(out)
        out = swish(out)
        return out
class Red(nn.Module):
    def __init__(self, equal_map):
        super(Red, self).__init__()
        self.equal_map = equal_map
        if equal_map == True:
            self.conv1 = nn.Conv2d(64, 64, 3, stride = 1, padding = 1)

        else:
            self.conv1 = nn.Conv2d(32, 64, 3, stride = 2, padding = 1)
        self.conv2 = nn.Conv2d(64, 64, 3, stride = 1, padding = 1)
        
        if self.equal_map == False:
            self.proj = nn.Conv2d(32, 64, 1, stride = 2)
        self.bn1 = nn.BatchNorm2d(64)
    def forward(self, x):
        def swish(pre):
            return pre * F.sigmoid(pre)
        out = self.conv1(x)
        # out = F.relu(self.bn1(out))
        out = swish(self.bn1(out))
        out = self.conv2(out)

        if self.equal_map == False:
            x = self.proj(x)

        out = x + out
        # out = F.relu(out)
        out = swish(out)
        return out
# define the SimpleNN mode;
class ResNet20(nn.Module):
    def __init__(self):
        super(ResNet20, self).__init__()
        #yellow
        self.yellow = nn.Conv2d(3, 16, 3, stride = 1, padding = 1)
        #blue
        self.blue = nn.Sequential(Blue(True), 
                                  Blue(True), 
                                  Blue(True))
        #green
        self.green = nn.Sequential(Green(equal_map = False), 
                                   Green(equal_map = True), 
                                   Green(equal_map = True))
        #red
        self.red = nn.Sequential(Red(False), 
                                 Red(True), 
                                 Red(True))
        self.fc = nn.Linear(256, 10)
        # self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):

        out = self.yellow(x)
        out = F.max_pool2d(out, 2)
        out = self.blue(out)
        out = self.green(out)
        out = self.red(out)

        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        # out = F.softmax(out, dim = 1)
        return out
 

In [2]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tools.dataset import CIFAR10

DATA_ROOT = "./data"
BATCH_SIZE = 100

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
    transform=transform_test
)

# do NOT shuffle your test data loader!!!!!!!!!!!!!!!!
# otherwise the order of samples will be messed up
# and your test accuracy is likely to drop to random guessing level
test_loader = DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

#########################################################
# use your model to generate predictions on test data
# and save the results into variable "results"
# "results" should be either a numpy array or a torch tensor with length of 10000

# initialize a resnet and load trained weights
net = ResNetCIFAR()
state_dict = torch.load('./saved_model/simplenn.pth') # change the path to your own checkpoint file
net.load_state_dict(state_dict)
net.cuda()

# remember to switch to eval mode whenever you are making inference
net.eval()

results = []
with torch.no_grad():
    for x in test_loader:
        results.append(
            net(x.cuda()).argmax(1)
        )

# convert results to numpy array
results = torch.cat(results).cpu().numpy()
assert len(results) == 10000

#########################################################
with open('predictions.csv', 'w') as fp:
    fp.write("Id,Label\n")
    for i in range(len(results)):
        fp.write("%d,%d\n" %(i, results[i]))

Interrupted during dataset downloading. Cleaning up...


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html